<a href="https://colab.research.google.com/github/rafelsiregar/imagemlp/blob/master/ML_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def sigmoid(x):
  return 1/(1+exp(-x))